In [ ]:
!pip install ffmpeg opencv-python opencv-python-headless basicsr facexlib gfpgan tqdm -q
!pip install -U git+https://github.com/xinntao/Real-ESRGAN.git -q

In [ ]:
!git clone https://github.com/xinntao/Real-ESRGAN.git
%cd Real-ESRGAN
!wget -P experiments/pretrained_models https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth
!pip install -r requirements.txt
!python setup.py develop
%cd ..

file_path = "/usr/local/lib/python3.11/dist-packages/basicsr/data/degradations.py"
with open(file_path, "r") as f:
    lines = f.readlines()
with open(file_path, "w") as f:
    for line in lines:
        if 'from torchvision.transforms.functional_tensor import rgb_to_grayscale' in line:
            f.write('from torchvision.transforms.functional import rgb_to_grayscale\n')
        else:
            f.write(line)



In [ ]:
import os, cv2, requests, subprocess, nest_asyncio
from fastapi import FastAPI, Form
from fastapi.responses import FileResponse
from pyngrok import ngrok
import uvicorn
import threading
import sys

nest_asyncio.apply()
app = FastAPI()

@app.post("/upscale/")
async def upscale_video(video_url: str = Form(...)):
    try:
        print("Downloading input video...")
        input_path = "/content/input.mp4"
        with open(input_path, "wb") as f:
            f.write(requests.get(video_url).content)

        os.makedirs("frames", exist_ok=True)
        os.system(f"ffmpeg -i {input_path} -qscale:v 1 frames/frame%04d.png"))
        print("Upscaling frames...")
        os.makedirs("frames_upscaled", exist_ok=True)
        result = subprocess.run([
            "python", "Real-ESRGAN/inference_realesrgan.py",
            "-n", "RealESRGAN_x4plus",
            "-i", "frames",
            "-o", "frames_upscaled"
        ], capture_output=True, text=True)
        print("Real-ESRGAN script output (stdout):")
        print(result.stdout)
        print("Real-ESRGAN script output (stderr):")
        print(result.stderr)

        output_video = "/content/upscaled_output_4k.mp4"
        os.system(f"ffmpeg -y -r 30 -start_number 1 -i frames_upscaled/frame%04d_out.png -vf scale=3840:2160 -c:v libx264 -crf 16 -preset slow -pix_fmt yuv420p {output_video}")

        print("Upscaling complete.")
        return FileResponse(output_video, media_type="video/mp4", filename="upscaled_output.mp4")

    except Exception as e:
        print("ERROR:", e)
        return {"error": str(e)}

In [ ]:
!ngrok kill

In [ ]:
!ngrok config add-authtoken ngrok_token_here

def run():
    uvicorn.run(app, host="0.0.0.0", port=8007)
threading.Thread(target=run).start()

public_url = ngrok.connect(8007)
print("Public URL:", public_url)

In [ ]:
import shutil, os

def clean_folder(path):
    if os.path.exists(path):
        shutil.rmtree(path)
    os.makedirs(path)

clean_folder("frames")
clean_folder("frames_upscaled")


In [ ]:
!curl -X POST "https://e8957ca2455a.ngrok-free.app/upscale/" \
  -F "video_url=https://res.cloudinary.com/dfmdmqz4k/video/upload/v1753926949/shortest_zaxyqt.mp4" \
  --output upscaled_output.mp4

In [ ]:
!fuser -k 8001/tcp  # kills any process using port 8004
